In [9]:
%run ../ModelSEEDpy/modelseedpy/biochem/modelseed_justification.py

model = justifyDB(msdb_path=".", primals_path="MSDB_justification_primals.json")

usage: modelseed_justification.py [-h] --msdb_path MSDB_PATH [--primals_path PRIMALS_PATH]
modelseed_justification.py: error: the following arguments are required: --msdb_path


SystemExit: 2

KeyError: 'H'

In [4]:
from modelseedpy.biochem import from_local
msdb = from_local(".")

In [5]:
troublesome_rxn = msdb.reactions.get_by_id("rxn05145")
troublesome_rxn.reaction

'cpd00001_0 + cpd00002_0 + cpd00009_1 --> cpd00008_0 + 2 cpd00009_0 + cpd00067_0'

In [1]:
primals = model.primal_values

NameError: name 'primals' is not defined